Compute a flag based on seed clustereing

In [1]:
import os
from pathlib import Path
import numpy as np
import requests
import pandas as pd

In [2]:
import json

In [3]:
from sbscorer.sblegos.TransactionAnalyser import TransactionAnalyser as txa

In [4]:
# Set path to data folder
current_dir = Path(os.getcwd())
DATA_DIR = os.path.join(current_dir.parent.parent, 'data-regen-rangers')
DATA_DIR_GITCOIN = os.path.join(current_dir.parent.parent, 'data-gitcoin')
GRANT_ROUND_ID = '0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc'
ODC_DATA_DIR = os.path.join(current_dir.parent.parent.parent, 'data')

PATH_TO_VOTES = os.path.join(DATA_DIR_GITCOIN, 'citizen-votes.csv')
# PATH_TO_GRANTS = os.path.join(DATA_DIR, "all-allo-rounds.csv")
# PATH_TO_PROJECTS = os.path.join(DATA_DIR, "projects_QmQurt.csv")

In [5]:
FOLDER_NAME = 'gr18'
CHAIN = 'optimism'
extract_all = True

# Set path to data folder
current_dir = Path(os.getcwd())
PATH_TO_EXPORT = os.path.join(current_dir.parent.parent, 'tx_data', FOLDER_NAME)
PATH_TO_GR18 = os.path.join(current_dir.parent.parent, FOLDER_NAME)
voter_tx_file_name = 'gg18_votes.csv'
passport_file_name = 'gg18_voter_passports.csv'

tx_file_name = 'gr_18_optimism_all_tx.csv'

In [6]:
votes = pd.read_csv(os.path.join(PATH_TO_GR18, voter_tx_file_name))
passports = pd.read_csv(os.path.join(PATH_TO_GR18, passport_file_name))

C:\Users\DELL Admin\AppData\Local\Temp\ipykernel_20584\1560736169.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  votes = pd.read_csv(os.path.join(PATH_TO_GR18, voter_tx_file_name))


In [7]:
# Load tx data
CONTRACT_CREATION_AD_NAME = '0x_contract_creation'


df_tx = pd.read_csv(os.path.join(PATH_TO_EXPORT, tx_file_name))

df_tx.loc[df_tx['to_address'].isna(), 'to_address'] = CONTRACT_CREATION_AD_NAME

assert df_tx.isna().sum().sum() == 0
df_tx.drop(columns=['__row_index'], inplace=True)

print(f'df transaction shape : {df_tx.shape}')

# add EOA to df_tx
df_tx_EOA_FROM = df_tx.copy()
df_tx_EOA_FROM['EOA'] = df_tx_EOA_FROM['from_address']

df_tx_EOA_TO = df_tx.copy()
df_tx_EOA_TO['EOA'] = df_tx_EOA_TO['to_address']

df_tx_eoa = pd.concat([df_tx_EOA_FROM, df_tx_EOA_TO])
df_tx_eoa = df_tx_eoa.loc[df_tx_eoa['EOA'] != CONTRACT_CREATION_AD_NAME, :]



df transaction shape : (6894506, 8)


In [8]:
df_tx_eoa.shape

(13787932, 9)

In [9]:
tx_analyser = txa(df_tx_eoa.copy(), votes['voter'].str.lower().unique())

print(f'Len of df tx: {len(tx_analyser.df_transactions)}')

Len of df tx: 6953161


In [10]:
df_seed = tx_analyser.df_seed_wallet
ar_un_seed = df_seed['from_address'].unique()

In [11]:
df_seed.isna().sum()

from_address    0
to_address      0
dtype: int64

In [12]:
len(ar_un_seed)

8687

We will consider that these wallet can't be flagged with the new seed clustering algorithm.

In [55]:
# load labels for EOA to know if it is a bridge or an exchange or something else
df_labels = pd.read_csv(os.path.join(ODC_DATA_DIR, 'address_labels_citizen.csv'), usecols=['address', 'tag', 'sub_type'])
df_labels_test = pd.read_csv(os.path.join(ODC_DATA_DIR, 'address_labels_citizen_test.csv'), usecols=['address', 'tag', 'sub_type'])
df_labels = pd.concat([df_labels, df_labels_test])
print(df_labels.shape)
df_labels.drop_duplicates(inplace=True)
print(df_labels.shape)

(105, 3)
(104, 3)


In [56]:
df_labels['cex_or_bridge'] = df_labels['tag'].apply(lambda x: True if x in ['cex_or_bridge', 'bridge', 'cex'] else False)

In [57]:
df_labels.columns

Index(['address', 'tag', 'sub_type', 'cex_or_bridge'], dtype='object')

In [58]:
df_seed.head()

,from_address,to_address
0,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,0x00000001985c6a08963eef53821c78aa84318b42
1,0x8052a1e6a7f64325ee3a7ce1c5e1cc39b22af712,0x00000002f32c0886ee65d68059fbdb76ef6a6996
2,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,0x000000064730640b7d670408d74280924883064f
3,0x646e9a6a66726d54fa1137df4448fda922c50f0d,0x00000009d94baf057697d988d2e3eec2fce81162
4,0x62b2f8888413cf2619f2eab477b811d989083f75,0x0000000a362b7ae7239e186b2bbeb943305c59ad


In [17]:
arr_diff_labels = np.setdiff1d(df_seed['from_address'].unique(), df_labels['address'].unique())

In [18]:
print(f'Number of addresses in labels but not in our known labels: {len(arr_diff_labels)}')

Number of addresses in labels but not in our known labels: 8619


We use the tag-address to get the list of addresses that are tagged as exchange or bridge.

In [20]:
arr_diff_labels

array(['0x00000000000cd56832ce5dfbcbff02e7ec639bc9',
       '0x00000002f32c0886ee65d68059fbdb76ef6a6996',
       '0x00000023f6b4ed7185e7b8928072a8bfec660ff3', ...,
       '0xffebf6747d8b46852e552d0ffebe71d47ac9fcae',
       '0xffec1b5f5a24b5fc2457763704191aa220c8bf78',
       '0xfff4900fa3c47ea46cc4063937c8547ecc4eb9b3'], dtype=object)

In [22]:
url_tag_api = 'http://localhost:8000/infer/optimism'
headers = {'Content-type': 'application/json'}
response = requests.post(url_tag_api, headers=headers, json={"address_list": arr_diff_labels.tolist()})



In [23]:
print(response.status_code)

200


In [ ]:
# write json to file
# json_string = json.dumps(json_res)
# with open('data.json', 'w') as f:
#     json.dump(json_string, f)

In [26]:
df_res = pd.read_json(response.json()['result'], orient='split')
df_res.head(2)

,eoa,prediction
0,0x08928dcb5bd0b58f8de51921eaade7da8146d259,0
1,0x156ddb799cc9ecb1a099f3fac2a3ce5affef266e,0


In [27]:
df_res.to_csv(os.path.join(ODC_DATA_DIR, 'seed_labels_gr18_raw.csv'), index=False)

In [28]:
# Concatenate the results with the known labels
# df_labels_all = pd.concat([df_labels.loc[:, ['address', 'cex_or_bridge']], df_res], ignore_index=True)
df_labels_all = df_res.rename(columns={'eoa': 'address', 'prediction': 'cex_or_bridge_raw'})

In [29]:
df_labels_all.head(2)

,address,cex_or_bridge_raw
0,0x08928dcb5bd0b58f8de51921eaade7da8146d259,0
1,0x156ddb799cc9ecb1a099f3fac2a3ce5affef266e,0


In [35]:
df_labels['cex_or_bridge'].sum()

32

In [30]:
df_seed.reset_index(inplace=True, drop=True)

In [31]:
df_labels_all['address'] = df_labels_all['address'].str.lower()

In [32]:
df_seed['from_address'] = df_seed['from_address'].str.lower()

In [38]:
# For each seed wallet check if it is a bridge or a cex and aggragate the results by seed wallet
df_seed_labels = df_seed.merge(df_labels_all, left_on='from_address', right_on='address', how='left')
df_seed_labels.isna().sum()

from_address             0
to_address               0
address              27326
cex_or_bridge_raw    27326
dtype: int64

In [39]:
df_seed_labels.dropna(subset=['cex_or_bridge_raw'], inplace=True)

In [43]:
df_seed_agg = df_seed_labels.groupby('from_address').agg({'cex_or_bridge_raw': 'sum'}).sort_values(by='cex_or_bridge_raw', ascending=False)

In [48]:
df_seed_agg[df_seed_agg['cex_or_bridge_raw'] > 1]

,cex_or_bridge_raw
from_address,
0x45a318273749d6eb00f5f6ca3bc7cd3de26d642a,87.0
0xd267a84a50c6993823c2c4b66610aca137812b13,12.0
0x030081c7d9c0cf6f3fdd3614a7c5bd6b4369934a,9.0
0xab75ca9ac66c0499d4ebf0b925777c7d282e015e,4.0
0x1eaae1b90bb9331d53a5fd835f2da0147e49f777,3.0
0x61f6bcc9f67a9dab6eff059f0d42d7da34b871a6,3.0
0x994841265cd8a18a710d839da4ca8000b0a121c5,3.0
0x827c83377cdc4c6bf6e8c5ca133f883890837230,3.0
0xc707ca69b2323a0703a11de784cb70417b0844e1,3.0


some manual work to verify that the addresses are indeed exchange or bridge.
We know that arround 10% of addresses are mislabelled.

From these addresses only the first one is a cex others are bots.

We then add that address to the list of labels and can now use the seed clustering algorithm.

In [59]:
new_df_cex = pd.DataFrame(columns=['address', 'cex_or_bridge'], data=[['0x45a318273749d6eb00f5f6ca3bc7cd3de26d642a', True]])
df_labels = pd.concat([df_labels, new_df_cex], ignore_index=True)

In [60]:
df_labels

,address,tag,sub_type,cex_or_bridge
0,0x4a1939dc1de524ff01980a911f0f4d65ce7a27ba,airdrop_master,NaN,False
1,0xeec428c18ff39c66162e39e79ed6eb8c790e43d7,cex_or_bridge,?,True
2,0x8f5075a0795864253e1f11313e5f98accf512fed,airdrop_master,NaN,False
3,0x9ce0dc49df0ab26690748e90a6f2c2ad9cec5fad,airdrop_master,NaN,False
4,0x5bdf85216ec1e38d6458c870992a69e38e03f7ef,cex,BitGet,True
...,...,...,...,...
100,0x54a3b587448e605f854e4274b50d4ac024ed3bfb,user,50k,False
101,0x1beb5a36f40f68eccd53899c94902e9905135456,bot,scam,False
102,0x1579e97c0c9619c61aef016d055b3d7de7f14bdb,airdrop_master,medium,False
103,0x27ee985d1e446ec71c277c89cd877ec4eeaa236c,cex,Binance?,True


In [64]:
df_count_seed = df_seed.groupby('from_address').count().sort_values(by='to_address', ascending=False)
df_count_seed.head(2)

,to_address
from_address,
0xacd03d601e5bb1b275bb94076ff46ed9d753435a,9105
0x80c67432656d59144ceff962e8faf8926599bcf8,6360


In [65]:
array_address_cex_or_bridge = df_labels[df_labels['cex_or_bridge']]['address'].unique()
print(len(array_address_cex_or_bridge))

33


In [70]:
df_count_seed['cex_or_bridge'] = df_count_seed.index.isin(array_address_cex_or_bridge)
df_count_seed.rename(columns={'to_address': 'EOA'}, inplace=True)
df_count_seed[~df_count_seed['cex_or_bridge']].head(20)


,EOA,cex_or_bridge
from_address,,
0x17f52a50227ea7aa6042eaae44c84a9a9d59d092,274,False
0xd7367bd1a720f42d9f4ba0043db89dc49cf80f6d,52,False
0xb4bb4a094338bc566e4bb0355c9de525f7c1f1b4,50,False
0x5b965a9cdcdb710d50967b0ac32e6341b72063a2,38,False
0xa3992a99b5838ababb2606a697be12384e5b2655,34,False
0xc6d7cba263bc5afb0ecc97820d8c6c6c9c92b0c2,24,False
0x27899fface558bde9f284ba5c8c91ec79ee60fd6,18,False
0x3047bf8e87f0786702b8e8265c3b04cedba920e3,18,False
0xce557bb2eeb69162116e60472890503b9da8c5a6,16,False


Here we compute some metric to see how many sybil could be flagged with the new seed clustering algorithm.
These is not the best approach because we should look at these suspicious addresses and their votes to see if they are sybil or not.

In [71]:
df_suspicous_seed = df_count_seed.loc[np.logical_and(~df_count_seed['cex_or_bridge'], df_count_seed['EOA'] > 2)]
print(df_suspicous_seed.head(2))
print(f'number of suspicious seeding wallets: {df_suspicous_seed.shape[0]}')
print(f'number of expected sybils: {df_suspicous_seed["EOA"].sum()}')

                                            EOA  cex_or_bridge
from_address                                                  
0x17f52a50227ea7aa6042eaae44c84a9a9d59d092  274          False
0xd7367bd1a720f42d9f4ba0043db89dc49cf80f6d   52          False
number of suspicious seeding wallets: 381
number of expected sybils: 2168


In [72]:
df_suspicous_seed = df_count_seed.loc[np.logical_and(~df_count_seed['cex_or_bridge'], df_count_seed['EOA'] > 1)]
print(f'number of suspicious seeding wallets: {df_suspicous_seed.shape[0]}')
print(f'number of expected sybils: {df_suspicous_seed["EOA"].sum()}')

number of suspicious seeding wallets: 1085
number of expected sybils: 3576


Above 3 seeding of voters we can expect the voter address to be a sybil. At 2 we may not be sure so the best way is to look at the votes.

In [75]:
df_seed['EOA'] = df_seed['to_address']

In [76]:
array_suspicous_seed = df_suspicous_seed.index.values
array_sus_seed_eoa = df_seed[df_seed['EOA'].isin(array_suspicous_seed)].EOA.unique()
array_sus_seed_from = df_seed[df_seed['from_address'].isin(array_suspicous_seed)].from_address.unique()
suspicious_voter = df_seed[np.logical_or(df_seed['EOA'].isin(array_suspicous_seed), df_seed['from_address'].isin(array_suspicous_seed))]['EOA'].unique()
print(f'number of suspicious voters: {len(suspicious_voter)}')
print(f'number of suspicious_seeder {len(array_suspicous_seed)}')

number of suspicious voters: 4157
number of suspicious_seeder 1085


In [77]:
df_seed_suspicious = df_seed[np.logical_or(df_seed['EOA'].isin(array_suspicous_seed), df_seed['from_address'].isin(array_suspicous_seed))].copy()

In [78]:
cluster = []
for group in df_seed_suspicious.groupby('from_address'):
    small_cluster = np.array([])
    if group[0] in array_address_cex_or_bridge:
        small_cluster = group[1]['EOA'].unique()
    else:
        small_cluster = np.append(group[1]['EOA'].unique(), group[0])
    cluster.append(small_cluster)

In [79]:
df_seed_suspicious.from_address.unique().shape[0]

1365

In [80]:
assert len(cluster) == df_seed_suspicious.from_address.unique().shape[0]
print(f'number of suspicious clusters: {len(cluster)} before merging')

number of suspicious clusters: 1365 before merging


Now we merge clusters that contains at least one common address

In [81]:
def merge_cluster(cluster):
    new_cluster = []
    cluster_merged = []
    for i in range(len(cluster)):
        if not i in cluster_merged:
            small_cluster = cluster[i]
            for j in range(i+1, len(cluster)):
                if len(np.intersect1d(small_cluster, cluster[j])) > 0:
                    small_cluster = np.append(small_cluster, cluster[j])
                    cluster_merged.append(j)
            new_cluster.append(np.unique(small_cluster))
    return new_cluster

In [82]:
def merge_cluster_full(cluster):
    print(len(cluster))
    new_cluster = merge_cluster(cluster)
    while len(new_cluster) != len(cluster):
        cluster = new_cluster
        new_cluster = merge_cluster(cluster)
        print(len(new_cluster))
    return new_cluster

In [83]:
new_cluster = merge_cluster_full(cluster)

1365
603
596
596


In [84]:
def max_length_cluster(cluster):
    max_length = 0
    for i in range(len(cluster)):
        if len(cluster[i]) > max_length:
            max_length = len(cluster[i])
    return max_length

print(f'max length of cluster: {max_length_cluster(new_cluster)}')

max length of cluster: 506


In [85]:
# minimum number of wallets seeded
df_seed[df_seed['from_address'].isin(array_suspicous_seed)].groupby('from_address').count()['EOA'].min()

2

In [86]:
def assign_cluster(address, cluster):
    for i in range(len(cluster)):
        if address in cluster[i]:
            return cluster[i]
    return np.nan

def assign_cluster_number(address, cluster):
    for i in range(len(cluster)):
        if address in cluster[i]:
            return i
    return -1

In [88]:
df_seed.drop(columns=['to_address'], inplace=True)

In [91]:
# Either the cluster corresponds to the seed wallet or the voter is the seed wallet
# we have to remove the case where the voter seed wallet is an exchange then we assign the voter address
df_seed['cluster_address'] = df_seed['EOA'].apply(lambda x: assign_cluster(x, new_cluster))
df_seed['cluster_number'] = df_seed['EOA'].apply(lambda x: assign_cluster_number(x, new_cluster))

In [92]:
df_seed['cluster_size'] = df_seed['cluster_address'].apply(lambda x: len(x) if isinstance(x, np.ndarray) else 0)

In [102]:
df_seed.drop_duplicates(subset=['cluster_number']).sort_values(by='cluster_size', ascending=False).head(10)

,from_address,EOA,cluster_address,cluster_number,cluster_size
43,0x719028736f10164c838ef129936779ed739312f2,0x002b5dfb3c71e1dc97a2e5a0a7f69f3e7b83f269,"[0x002b5dfb3c71e1dc97a2e5a0a7f69f3e7b83f269, 0...",3,506
243,0xb5c69e6dcb418d902598cdc298717ff7c484a805,0x0165efba0d7381cb9c10509df5dca9229c401ae6,"[0x0165efba0d7381cb9c10509df5dca9229c401ae6, 0...",18,441
302,0x17f52a50227ea7aa6042eaae44c84a9a9d59d092,0x01eec0c7faf859582b68898868af7ed875e48d40,"[0x01eec0c7faf859582b68898868af7ed875e48d40, 0...",86,281
84,0xb4bb4a094338bc566e4bb0355c9de525f7c1f1b4,0x0065981684b5e02f95283b8ce1916b8981206a3d,"[0x0065981684b5e02f95283b8ce1916b8981206a3d, 0...",38,117
124,0x01ccb548cb303f1c552eeb660fec6f3d7876c0a0,0x00aefcafae46fd245fb8ca555fe30f4713760579,"[0x00aefcafae46fd245fb8ca555fe30f4713760579, 0...",6,88
133,0xaa1062ff44cb41b9aac4879dda6e7c21c1e8aa3b,0x00bd3e58009601c77043672a8271451b18c9d6c2,"[0x00bd3e58009601c77043672a8271451b18c9d6c2, 0...",2,64
314,0x0628ffe7f4e299e817761d05c0c7e89513ba308a,0x020109398ac397da015c903aec703b2de94cf1c0,"[0x020109398ac397da015c903aec703b2de94cf1c0, 0...",23,63
347,0xd7367bd1a720f42d9f4ba0043db89dc49cf80f6d,0x023b38379fa2a4f44c5991b655ab79834f2786a9,"[0x023b38379fa2a4f44c5991b655ab79834f2786a9, 0...",206,61
2241,0x5b965a9cdcdb710d50967b0ac32e6341b72063a2,0x0f2261d0374a202466023ff71105674b1238f3b3,"[0x0f2261d0374a202466023ff71105674b1238f3b3, 0...",300,39
1628,0xa3992a99b5838ababb2606a697be12384e5b2655,0x0afbea5597875c33047ab0004575f636e652c49e,"[0x0afbea5597875c33047ab0004575f636e652c49e, 0...",465,35


In [105]:
np.concatenate(new_cluster)

array(['0x00000002f32c0886ee65d68059fbdb76ef6a6996',
       '0x548be6667e92e742dbd852662349ae74bb39a215',
       '0x6f5da981982011f1ac2ec345b5c2681c16298f2e', ...,
       '0xfda9c289eb901d495dec764050f9d596eb64cbbe',
       '0x8ae339f9a9fddf930cf10d7fcb4abba417cd50fd',
       '0xff935e24d108e316484d07bbd9d7d6469677c23e'], dtype=object)

In [106]:
# verify no cluster address from a cex
np.intersect1d(np.unique(np.concatenate(new_cluster)), array_address_cex_or_bridge)

array([], dtype=object)

In [107]:
df_seed.to_csv(os.path.join(PATH_TO_GR18, 'seed_cluster_gr18.csv'), index=False)

In [112]:
df_seed.dropna(subset=['cluster_address']).to_csv(os.path.join(PATH_TO_GR18, 'seed_cluster_only_gr18.csv'), index=False)

In [108]:
df_seed

,from_address,EOA,cluster_address,cluster_number,cluster_size
0,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,0x00000001985c6a08963eef53821c78aa84318b42,NaN,-1,0
1,0x8052a1e6a7f64325ee3a7ce1c5e1cc39b22af712,0x00000002f32c0886ee65d68059fbdb76ef6a6996,"[0x00000002f32c0886ee65d68059fbdb76ef6a6996, 0...",0,4
2,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,0x000000064730640b7d670408d74280924883064f,NaN,-1,0
3,0x646e9a6a66726d54fa1137df4448fda922c50f0d,0x00000009d94baf057697d988d2e3eec2fce81162,NaN,-1,0
4,0x62b2f8888413cf2619f2eab477b811d989083f75,0x0000000a362b7ae7239e186b2bbeb943305c59ad,NaN,-1,0
...,...,...,...,...,...
38116,0x2d2cc0eb095e43204e0c087e07dbf95909650939,0xfff9e8387f6ea2073d1c38b9571fe4aa55fd3ad8,NaN,-1,0
38117,0xdf362413e0125a93c57c31472f5cc02bebb99306,0xfffb43b6b6048adc32ded4fcedc429b17e7ea72d,NaN,-1,0
38118,0x053b5ac05c40a4bb348a6047da6979dd34f2822d,0xfffb609cda6131468b17391e6e4f3124cf180333,"[0x053b5ac05c40a4bb348a6047da6979dd34f2822d, 0...",24,6
38119,0x4b440d72a15d2c367a6e471ba6fc8c61d1a0f4a5,0xfffbbbc0504cc643ef14cd89269c4d9cbce428c9,NaN,-1,0
